# Compute deformations

This notebook performs a geodesic interpolation between each pair of consecutive meshes.

The meshes are unparameterized surfaces, on which the generalized elastic metrics are used.

The geodesic itself is saved as new meshes. The geodesic shows how a mesh smoothly deforms onto the next mesh of the time-series.

This notebook should be run on the server where a GPU is available.

## Setup

In [1]:
import os
import subprocess

gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
print("Working directory: ", os.getcwd())

import warnings

warnings.filterwarnings("ignore")

import sys

sys_dir = os.path.dirname(os.getcwd())
sys.path.append(sys_dir)
h2_surfacematch_dir = os.path.join(sys_dir, "H2_SurfaceMatch")
sys.path.append(h2_surfacematch_dir)
sys.path.append("..")

print("Added directories to syspath:")
print(sys_dir)
print(h2_surfacematch_dir)

Working directory:  /home/nmiolane/code/my28brains
Added directories to syspath:
/home/nmiolane/code
/home/nmiolane/code/H2_SurfaceMatch


## Imports

In [14]:
import glob
import time

import numpy as np
import torch
import trimesh
from sklearn.decomposition import PCA

import my28brains.my28brains.io as io

In [10]:
import H2_match as gm
import H2_SurfaceMatch.utils.input_output
import H2_SurfaceMatch.utils.utils
import SRNF_match

use_cuda = 1
torchdeviceId = torch.device("cuda:0") if use_cuda else "cpu"
torchdtype = torch.float32

print(torchdeviceId)

cuda:0


## Overview

Currently, each mesh of the time-series has a different number of vertices and even if they had the same nnumber of vertices, these vertices would not correspond across meshes.

We say that the surface mesh are "unparameterized" in the sense that they are not parameterized in the same way.

We would like to use geomstats code to perform statistics on these surface meshes, however geomstats code only posssesses the parameterized version for now. 

Thus, we will reparameterize the curves so that they have the same number of vertices, and matching vertices and faces.

To this aim:
- We consider the mesh parameterization of the first mesh of each time-series: i.e., the. base mesh `B`.
- We will register the base mesh `B` to each of the other mesh `M` and get a transformation `phi`.
- We will apply the ransformation `phi` to the base mesh `B`.


--> This will give a new mesh `phi * B` that looks like M but is parameterized like B.

Note: if the function decimate_mesh is precise, we can use it to have meshes with the same number of vertices...

## Define Dirs

In [4]:
CENTERED_MESHES_DIR = os.path.join(os.getcwd(), "data", "centered_meshes")
print("CENTERED_MESHES_DIR: ", CENTERED_MESHES_DIR)

CENTERED_NONDEGENERATE_MESHES_DIR = os.path.join(
    os.getcwd(), "data", "centered_nondegenerate_meshes"
)
print("CENTERED_NONDEGENERATE_MESHES_DIR: ", CENTERED_NONDEGENERATE_MESHES_DIR)
if not os.path.exists(CENTERED_NONDEGENERATE_MESHES_DIR):
    os.makedirs(CENTERED_NONDEGENERATE_MESHES_DIR)

CENTERED_MESHES_DIR:  /home/nmiolane/code/my28brains/data/centered_meshes
CENTERED_NONDEGENERATE_MESHES_DIR:  /home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes


## Remove degenerate faces in all meshes

In [5]:
hemisphere = "left"
structure_id = -1
string_base = os.path.join(
    CENTERED_MESHES_DIR, f"{hemisphere}_structure_{structure_id}_sub-01_ses-**.ply"
)
paths = sorted(glob.glob(string_base))

print(
    f"Found {len(paths)} ply files for {hemisphere} hemisphere and anatomical structure {structure_id}:"
)
for path in paths:
    print(path)

Found 27 ply files for left hemisphere and anatomical structure -1:
/home/nmiolane/code/my28brains/data/centered_meshes/left_structure_-1_sub-01_ses-02.ply
/home/nmiolane/code/my28brains/data/centered_meshes/left_structure_-1_sub-01_ses-03.ply
/home/nmiolane/code/my28brains/data/centered_meshes/left_structure_-1_sub-01_ses-04.ply
/home/nmiolane/code/my28brains/data/centered_meshes/left_structure_-1_sub-01_ses-05.ply
/home/nmiolane/code/my28brains/data/centered_meshes/left_structure_-1_sub-01_ses-06.ply
/home/nmiolane/code/my28brains/data/centered_meshes/left_structure_-1_sub-01_ses-07.ply
/home/nmiolane/code/my28brains/data/centered_meshes/left_structure_-1_sub-01_ses-08.ply
/home/nmiolane/code/my28brains/data/centered_meshes/left_structure_-1_sub-01_ses-09.ply
/home/nmiolane/code/my28brains/data/centered_meshes/left_structure_-1_sub-01_ses-10.ply
/home/nmiolane/code/my28brains/data/centered_meshes/left_structure_-1_sub-01_ses-11.ply
/home/nmiolane/code/my28brains/data/centered_meshes/

We will use the centered meshes, since they seemed more aligned than the registered meshes.

In [7]:
for path in paths:
    print(f"\tLoad mesh from path: {path}")
    mesh = trimesh.load(path)
    new_vertices, new_faces = io.remove_degenerate_faces(mesh.vertices, mesh.faces)
    new_mesh = trimesh.Trimesh(vertices=new_vertices, faces=new_faces)

    ply_path = os.path.join(CENTERED_NONDEGENERATE_MESHES_DIR, os.path.basename(path))
    print(ply_path)
    io.write_trimesh_to_ply(new_mesh, ply_path)

	Load mesh from path: /home/nmiolane/code/my28brains/data/centered_meshes/left_structure_-1_sub-01_ses-02.ply
/home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-02.ply
Writing mesh at /home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-02.ply...
	Load mesh from path: /home/nmiolane/code/my28brains/data/centered_meshes/left_structure_-1_sub-01_ses-03.ply
/home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-03.ply
Writing mesh at /home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-03.ply...
	Load mesh from path: /home/nmiolane/code/my28brains/data/centered_meshes/left_structure_-1_sub-01_ses-04.ply
/home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-04.ply
Writing mesh at /home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-04.ply...
	

# Create sequence of ply showing smooth deformation by geodesic interpolation

In [23]:
hemisphere = "left"
structure_id = -1
string_base = os.path.join(
    CENTERED_NONDEGENERATE_MESHES_DIR,
    f"{hemisphere}_structure_{structure_id}_sub-01_ses-**.ply",
)
paths = sorted(glob.glob(string_base))

print(
    f"Found {len(paths)} ply files for {hemisphere} hemisphere and anatomical structure {structure_id}:"
)
for path in paths:
    print(path)

Found 27 ply files for left hemisphere and anatomical structure -1:
/home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-02.ply
/home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-03.ply
/home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-04.ply
/home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-05.ply
/home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-06.ply
/home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-07.ply
/home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-08.ply
/home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-09.ply
/home/nmiolane/code/my28brains/data/centered_nondegenerate_meshes/left_structure_-1_sub-01_ses-10.ply
/home/nmiolane

In [24]:
start_paths = paths[:-1]
end_paths = paths[1:]

Can we force the interp on one given GPU? it seems, in any case, that it is using all CPUs... thus might not help

In [25]:
a0 = 0.01
a1 = 100
b1 = 100
c1 = 0.2
d1 = 0.01
a2 = 0.01


param1 = {
    "weight_coef_dist_T": 10**1,
    "weight_coef_dist_S": 10**1,
    "sig_geom": 0.4,
    "max_iter": 2000,
    "time_steps": 2,
    "tri_unsample": True,
    "index": 0,
}

param2 = {
    "weight_coef_dist_T": 10**2,
    "weight_coef_dist_S": 10**2,
    "sig_geom": 0.3,
    "max_iter": 1000,
    "time_steps": 2,
    "tri_unsample": False,
    "index": 1,
}

param3 = {
    "weight_coef_dist_T": 10**3,
    "weight_coef_dist_S": 10**3,
    "sig_geom": 0.2,
    "max_iter": 1000,
    "time_steps": 2,
    "tri_unsample": False,
    "index": 1,
}

param4 = {
    "weight_coef_dist_T": 10**4,
    "weight_coef_dist_S": 10**4,
    "sig_geom": 0.1,
    "max_iter": 1000,
    "time_steps": 3,
    "tri_unsample": True,
    "index": 1,
}

param5 = {
    "weight_coef_dist_T": 10**5,
    "weight_coef_dist_S": 10**5,
    "sig_geom": 0.1,
    "max_iter": 1000,
    "time_steps": 4,
    "tri_unsample": False,
    "index": 2,
}

param6 = {
    "weight_coef_dist_T": 10**6,
    "weight_coef_dist_S": 10**6,
    "sig_geom": 0.05,
    "max_iter": 1000,
    "time_steps": 5,
    "tri_unsample": False,
    "index": 2,
}


paramlist = [param1, param2, param3, param4, param5, param6]

In [26]:
GEODESICS_DIR = os.path.join(
    os.getcwd(), "data", "geodesics"
)
print("GEODESICS_DIR: ", GEODESICS_DIR)
if not os.path.exists(GEODESICS_DIR):
    os.makedirs(GEODESICS_DIR)

GEODESICS_DIR:  /home/nmiolane/code/my28brains/data/geodesics


### Function performing one interpolation

In [29]:
def _geodesic_interp(i_pair):
    """Auxiliary function that will be run in parallelon different GPUs."""
    start_time = time.time()
    start_path = start_paths[i_pair]
    end_path = end_paths[i_pair]

    [VS, FS, FunS] = H2_SurfaceMatch.utils.input_output.loadData(start_path)
    VS = VS / 10
    [VS, FS] = H2_SurfaceMatch.utils.input_output.decimate_mesh(VS, FS, int(FS.shape[0] / 4))
    sources = [[VS, FS]]
    
    [VT, FT, FunT] = H2_SurfaceMatch.utils.input_output.loadData(end_path)
    VT = VT / 10
    [VT, FT] = H2_SurfaceMatch.utils.input_output.decimate_mesh(VT, FT, int(FT.shape[0] / 4))
    targets = [[VT, FT]]

    source = sources[0]
    target = targets[0]

    geod, F0 = gm.H2MultiRes(
        source, target, a0, a1, b1, c1, d1, a2, resolutions=2, paramlist=paramlist
    )
    comp_time = time.time() - start_time
    print(f"Geodesic interpolation {i_pair} took: {comp_time / 60:.2f} minutes.")
    
    # We use the start mesh as the basename for the ply files
    ply_prefix = os.path.join(GEODESICS_DIR, os.path.basename(start_path))

    for i_time in range(geod.shape[0]):
        H2_SurfaceMatch.utils.input_output.plotGeodesic(
            [geod[i_time]],
            F0,
            stepsize=5,
            file_name=ply_prefix + "{}".format(i_time),
            axis=[0, 1, 0],
            angle=-1 * np.pi / 2,
        )
    print(f"Geodesic interpolation {i_pair} saved to: {geod_dir}.")

In [30]:
for i_pair in range(len(start_paths)):
    print(f"\n\n -------> Geodesic interpolation for pair: {i_pair}/{len(start_paths)}")
    _geodesic_interp(i_pair)



 -------> Geodesic interpolation for pair: 0/26
before starting: Vertices then Faces for S then T
(4758, 3) (9596, 3)
(4504, 3) (9056, 3)
Resol 0: Vertices for S then T
FS decimation target: 599
(1205, 3) (2398, 3)
FT decimation target: 566
(1137, 3) (2264, 3)
Resol 1: Vertices for S then T
FS decimation target: 149
(320, 3) (598, 3)
FT decimation target: 141
(308, 3) (566, 3)
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         1920     M =           10


 This problem is unconstrained.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F

 ############ Iteration 0:
F0.shape (598, 3)
geod.shape: (2, 320, 3)
in  triupsample:
iteration 0 in range(N) to subdivice mesh
 1920   1186   1241      1     0     0   4.451D+01   3.469D+02

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
iteration 1 in range(N) to subdivice mesh
len(geod): 2
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         7266     M =           10



 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.
 This problem is unconstrained.



 ############ Iteration 1:
F0.shape (2392, 3)
geod.shape: (2, 1211, 3)

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 7266      2     45      2     0     0         NaN         NaN

ABNORMAL_TERMINATION_IN_LNSRCH                              
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         7266     M =           10

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point


...


           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
50736      1     21      1     0     0         NaN         NaN

ABNORMAL_TERMINATION_IN_LNSRCH                              

 ############ Iteration 5:
F0.shape (8544, 3)
geod.shape: (4, 4228, 3)
in timesteps:



 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        38052     M =           10


 This problem is unconstrained.


_LinAlgError: linalg.inv: (Batch element 1434): The diagonal element 2 is zero, the inversion could not be completed because the input matrix is singular.